In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [70]:
#db Connection String
# Create a database connection
user = 'postgres'
host = 'xx.xxx.xx.xxx'
dbname = 'mimic'
schema = 'mimiciii'
password ='xxxx'
port = '5432'

In [71]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host,port=port,password=password)
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))

In [ ]:
# Get length of stay from the icustays table
query = \
"""
SELECT * FROM admission_details
order by subject_id, admittime,intime;
"""

df_admission_details = pd.read_sql_query(query,con)
df_admission_details.to_csv('../data/admission_details_2.csv',index=False)

In [2]:
df_admission_details=pd.read_csv('../data/admission_details_2.csv')
df_admission_details.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,icustay_id,intime,outtime,last_wardid,first_careunit,last_careunit,age,gender,marital_status,insurance
0,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,243653,2138-07-17 21:20:07,2138-07-17 23:32:21,56,NICU,NICU,0.002175,M,NaN,Private
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,211552,2101-10-20 19:10:11,2101-10-26 20:43:09,12,MICU,MICU,76.526788,M,MARRIED,Medicare
2,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,294638,2191-03-16 00:29:31,2191-03-17 16:46:31,52,MICU,MICU,47.845044,F,SINGLE,Private
3,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,214757,2103-02-02 06:04:24,2103-02-02 08:06:00,56,NICU,NICU,0.000515,M,NaN,Private
4,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,228232,2175-05-30 21:30:54,2175-06-03 13:39:54,33,SICU,SICU,65.940670,F,MARRIED,Medicare


In [3]:
#Add column to show if readmitted
readmit_data=df_admission_details.assign(IsReadmitted=0)

In [4]:
#convert dates from string to datetime
readmit_data['admittime'] = readmit_data['admittime'].astype('datetime64[ns]')
readmit_data['dischtime'] = readmit_data['dischtime'].astype('datetime64[ns]')
readmit_data['intime'] = readmit_data['intime'].astype('datetime64[ns]')

#for missing outtime, assuming icu out time was same as discharged time
readmit_data.loc[readmit_data['outtime'].isnull(),'outtime'] = readmit_data['dischtime']
readmit_data['outtime'] = readmit_data['outtime'].astype('datetime64[ns]')

In [5]:
#total number of admissions
num_adms = readmit_data.shape[0]

for idx in range(1,num_adms):
    if readmit_data.subject_id[idx] == readmit_data.subject_id[idx - 1]:
        

        
        '''
        prev_dischtime = readmit_data.dischtime[idx-1]
        curr_admittime = readmit_data.admittime[idx] 
        
        readmit_time = curr_admittime - prev_dischtime
        if readmit_time.days <0:
            print(readmit_data.subject_id[idx])
        '''
        #previous icu discharge time
        prev_outtime = readmit_data.outtime[idx-1]
        #current icu admit time
        curr_intime = readmit_data.intime[idx] 
        
        readmit_time = curr_intime - prev_outtime
        if readmit_time.days <31:
            readmit_data.loc[idx-1, 'IsReadmitted'] = 1

In [10]:
readmit_data.to_csv('../data/readmits_details.csv',index=False)

In [8]:
df_readmission = pd.read_csv('../data/readmits_details.csv')

In [4]:
df_readmission.shape

(61532, 16)

In [12]:
#removing ppl less than 15 years of age
df_readmission=df_readmission[(df_readmission.age>=15)]

In [13]:
#removing ppl more than 90 years of age
df_readmission=df_readmission[(df_readmission.age<=90)]

In [14]:
df_readmission.shape

(50710, 16)

In [15]:
df_readm_clean=df_readmission.drop(['deathtime','admittime','dischtime','intime','outtime'], axis=1)

In [16]:
df_readm_clean.shape

(50710, 11)

In [17]:
df_readm_clean_2= df_readm_clean.sort_values(['subject_id', 'hadm_id','IsReadmitted'],ascending=[True,True, False])

In [38]:
df_readm_clean_2.shape

(50710, 11)

In [42]:
sum(df_readm_clean_2['IsReadmitted'])*100/len(df_readm_clean['IsReadmitted'])

11.778741865509762

In [18]:
df_readm_clean_2.head()

,subject_id,hadm_id,icustay_id,last_wardid,first_careunit,last_careunit,age,gender,marital_status,insurance,IsReadmitted
1,3,145834,211552,12,MICU,MICU,76.526788,M,MARRIED,Medicare,0
2,4,185777,294638,52,MICU,MICU,47.845044,F,SINGLE,Private,0
4,6,107064,228232,33,SICU,SICU,65.940670,F,MARRIED,Medicare,0
8,9,150750,220597,15,MICU,MICU,41.790226,M,NaN,Medicaid,0
10,11,194540,229441,57,SICU,SICU,50.148292,F,MARRIED,Private,0


In [19]:
df_readm_clean_2.to_csv('../data/clean_readm_details.csv',index=False)

In [20]:
df_readm_clean_3 = pd.read_csv('../data/clean_readm_details.csv')

In [62]:

from sklearn.model_selection import train_test_split
orig, subsample = train_test_split(df_readm_clean_3, test_size=10/500, random_state=20)
sum(subsample['IsReadmitted'])*100/len(subsample['IsReadmitted'])

11.822660098522167

In [63]:
subsample.shape

(1015, 11)

In [64]:
sum(subsample['IsReadmitted'])*100/len(subsample['IsReadmitted'])

11.822660098522167

In [66]:
subsample.to_csv('../data/subsample.csv',index=False)

In [67]:
subsample=subsample.reset_index(drop=True)

In [68]:
subsample['icustay_id'].to_csv('../data/icustay_ids.csv')

In [72]:
# Get length of stay from the icustays table
query = \
"""
SELECT * FROM charts_details;
"""

df_charts_details = pd.read_sql_query(query,con)
df_charts_details.to_csv('../data/df_charts_details_2.csv',index=False)

In [73]:
df_charts_details.shape

(6282992, 5)

In [79]:
from collections import Counter
cntr = Counter(df_charts_details['itemid'])
freq_item_ids=[item_id for item_id,_ in cntr.most_common(200)]
#print(Counter(df_charts_details['itemid']))

In [83]:
df_charts_subset = df_charts_details.loc[df_charts_details['itemid'].isin(freq_item_ids)]

In [88]:
len(set(df_charts_subset.icustay_id))

996

In [89]:
df_charts_subset.to_csv('../data/df_charts_subset.csv',index=False)

In [12]:
from datetime import datetime
#headers = ['icustay_id', 'itemid','charttime'] 

#dtypes = [int,int,datetime.datetime] 

df_charts_subset = pd.read_csv('../data/df_charts_subset.csv',dtype={"itemid": int, "icustay_id": int})

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
df_charts_subset['charttime'] = df_charts_subset['charttime'].astype('datetime64[ns]')

In [19]:
icu=set(df_charts_subset.icustay_id)

In [21]:
len(icu)

996

In [25]:
subsample=pd.read_csv('../data/subsample.csv')

In [23]:
df_new_subset = subsample.loc[subsample['icustay_id'].isin(icu)]

In [27]:
df_new_subset.shape

(996, 11)

In [28]:
sum(df_new_subset['IsReadmitted'])*100/len(df_new_subset['IsReadmitted'])

11.947791164658634